In [7]:
#imports
import pandas as pd
import requests
import pprint
import pycountry
import json

In [8]:
fred_api_key = 'c98f06b9ff696165b5171fbfe500692f'

# create an empty list to store the country names
country_names = []

# iterate over all the countries and append their names to the list
for country in pycountry.countries:
    country_names.append(country.name)

# loop through each country and get the series code
usable_countries = []  # create a new list to store countries with available CPI data
for country in country_names:
    # format the search query for FRED
    query = f"Population, Total for {country}"
    url = f"https://api.stlouisfed.org/fred/series/search?search_text={query}&api_key={fred_api_key}&file_type=json"
    # make a request to FRED API with the search query
    response = requests.get(url).json()
    # extract the series ID if the search returned results
    # check if there are any series codes returned
    if len(response['seriess']) > 0:
        series = response['seriess'][0]
        id = series['id']
        usable_countries.append([country,id])
        print(f'Got series code for {country}')
    else:
        print(f'No series code for {country}')
            

Got series code for Aruba
Got series code for Afghanistan
Got series code for Angola
No series code for Anguilla
No series code for Åland Islands
Got series code for Albania
Got series code for Andorra
Got series code for United Arab Emirates
Got series code for Argentina
Got series code for Armenia
Got series code for American Samoa
No series code for Antarctica
No series code for French Southern Territories
Got series code for Antigua and Barbuda
Got series code for Australia
Got series code for Austria
Got series code for Azerbaijan
Got series code for Burundi
Got series code for Belgium
Got series code for Benin
No series code for Bonaire, Sint Eustatius and Saba
Got series code for Burkina Faso
Got series code for Bangladesh
Got series code for Bulgaria
Got series code for Bahrain
Got series code for Bahamas
Got series code for Bosnia and Herzegovina
No series code for Saint Barthélemy
Got series code for Belarus
Got series code for Belize
Got series code for Bermuda
Got series co

KeyError: 'seriess'

In [9]:
# create a list of dictionaries to store the results
results = []

# loop through each usable country and get the CPI data
for country, series_code in usable_countries:
    url = f"https://api.stlouisfed.org/fred/series/observations?series_id={series_code}&api_key={fred_api_key}&file_type=json"
    response = requests.get(url)
    data = response.json()
    # loop through each observation in the data and add it to the results list
    for observation in data['observations']:
        date = observation['date']
        population = observation['value']
        if date >= '2018' and date <= '2022':
            results.append({
                "country": country,
                "series_code": series_code,
                "date": date,
                "Population": population
            })


# create a DataFrame from the results list
population_df = pd.DataFrame(results)
population_df.drop('series_code', axis=1, inplace=True)
population_df

,country,date,Population
0,Aruba,2018-01-01,105962
1,Aruba,2019-01-01,106442
2,Aruba,2020-01-01,106585
3,Aruba,2021-01-01,106537
4,Afghanistan,2018-01-01,.
...,...,...,...
557,Kenya,2019-01-01,62.943
558,Kenya,2020-01-01,62.675
559,Cambodia,2018-01-01,2.435
560,Cambodia,2019-01-01,2.402


In [10]:
# Save dataframe to CSV file
population_df.to_csv('population.csv', index=False)